### Production Features Pipeline - CSV Version

This notebook is run daily from a Github Action. 

1. It scrapes the results from the previous day's games, performs feature engineering, and saves the results back to a csv file. This is an alternative version of the pipeline that DOES NOT utilize the Hopsworks.ai Feature Store and is less dependent on other platforms.

2. It scrapes the upcoming games for today, and saves the blank records back into the csv file so that they can be accessed by the model for the prediction.

**Note:**
There are two options for webscraping in this notebook. 
Set the 'WEBSCRAPER' variable to either 'SCRAPINGANT' or 'SELENIUM' to choose which version to run.

1. SCRAPINGANT: Uses a webscraping service with a Python API, ScrapingAnt, which handles all the proxy server issues, but does require an account. The free account allows for 1000 page requests, which is more than enough for this project. Proxies are required when running this notebook from a Github Action or otherwise key data will fail to be scraped from NBA.com. 

2. SELENIUM: This option does not currently integrate proxy servers into the webscraping process, which can cause issues when scraping from certain locations, in particular Github Actions. For occasional use from local machines, this option may work fine, but you may need to setup a proxy server.

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
#WEBSCRAPER = 'SELENIUM'

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  #for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir('..') 

 
from src.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.dashboard_processing import (
    NBADataProcessor,
)

from src.google_drive_utils import (
    upload_to_drive,
)

DATAPATH = Path(r'data')
GOOGLE_FOLDER_ID = "1y5AfF3KZ8FGzxr2pyuncXJpKWEa5j-CL"

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

#try:
#    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
#except:
#    raise Exception('Set environment variable HOPSWORKS_API_KEY')


# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == 'SCRAPINGANT':
    try:
        SCRAPINGANT_API_KEY = os.environ['SCRAPINGANT_API_KEY']
    except:
        raise Exception('Set environment variable SCRAPINGANT_API_KEY')
    driver = None
    
elif WEBSCRAPER == 'SELENIUM':
    driver = activate_web_driver('chromium')
    SCRAPINGANT_API_KEY = ""
    



**Scrape New Completed Games and Format Them**

In [4]:


df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print('No new games to process')

    # determine what season we are in currently
    today = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    if today.month >= 10:
        SEASON = today.year
    else:
        SEASON = today.year - 1
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new['SEASON'].max()

    df_new




Current month is 05
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Playoffs&DateFrom=05/01/25&DateTo=05/08/25
0     1610612738
1     1610612752
2     1610612760
3     1610612743
4     1610612754
5     1610612739
6     1610612750
7     1610612744
8     1610612752
9     1610612738
10    1610612760
11    1610612743
12    1610612754
13    1610612739
14    1610612745
15    1610612744
16    1610612746
17    1610612743
18    1610612744
19    1610612745
20    1610612752
21    1610612765
22    1610612746
23    1610612743
dtype: object
HOME_TEAM_ID 0     1610612738
1     1610612760
2     1610612739
3     1610612750
4     1610612738
5     1610612760
6     1610612739
7     1610612745
8     1610612743
9     1610612744
10    1610612765
11    1610612746
Name: HOME_TEAM_ID, dtype: object
GAME_ID 0     0042400212
1     0042400222
2     0042400202
3     0042400231
4     0042400211
5     0042400221
6     0042400201
7     0042400157
8     0042400177
9     0042400156
10    0042400126
11    0

**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(SCRAPINGANT_API_KEY, driver)

if matchups is None:
    print('No games today')
else:
    print(matchups)
    print(game_ids)


Wed
Thu
[['1610612744', '1610612750']]
['42400232']


**Close Webdriver**

In [6]:
if WEBSCRAPER == 'SELENIUM':
    driver.close() 

**Check if anything is going on in the season**

In [7]:
if (df_new.empty) and (matchups is None):
    print('No new games to process')
    #exit()
    

**Create Rows for Today's Games with Empty Stats**

In [8]:
# reformat today's matchups to the new games dataframe

if matchups is None:
    print('No games going on. Nothing to do.')
    #exit()    

else:

    df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
    for i, matchup in enumerate(matchups):
        game_details = {'HOME_TEAM_ID': matchup[1], 
                        'VISITOR_TEAM_ID': matchup[0], 
                        'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                        'GAME_ID': int(game_ids[i]),                       
                        'SEASON': SEASON,
                        } 
        game_details_df = pd.DataFrame(game_details, index=[i])
        # append to new games dataframe
        df_today = pd.concat([df_today, game_details_df], ignore_index = True)

    #blank rows will be filled with 0 to prevent issues with feature engineering
    df_today = df_today.fillna(0) 

    df_today



**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [9]:


df_old = pd.read_csv(DATAPATH / 'games.csv')

df_old


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30997,2025-05-02 00:00:00,42400156.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,107.0,41.100,81.800,...,26.0,42.0,NaN,115.0,44.900,71.700,40.000,23.0,46.0,0
30998,2025-05-01 00:00:00,42400126.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,113.0,46.900,84.800,...,25.0,36.0,NaN,116.0,47.700,80.000,33.300,18.0,43.0,0
30999,2025-05-01 00:00:00,42400176.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,111.0,50.000,86.700,...,23.0,35.0,NaN,105.0,52.400,66.700,40.700,24.0,39.0,1
31000,2025-04-30 00:00:00,42400155.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,131.0,55.100,84.200,...,23.0,39.0,NaN,116.0,41.700,68.200,34.100,25.0,49.0,1


**Update Yesterday's Matchup Predictions with New Final Results**

In [10]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


# one approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)


# save the new games to the database
df_old.to_csv(DATAPATH / 'games.csv', index=False)

df_old

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-03-12,22101005.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,104.0,0.398,0.760,...,23.0,53.0,1.610613e+09,113.0,0.422,0.875,0.357,21.0,46.0,0
1,2022-03-12,22101006.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,101.0,0.443,0.933,...,20.0,46.0,1.610613e+09,91.0,0.419,0.824,0.208,19.0,40.0,1
2,2022-03-12,22101007.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,108.0,0.412,0.813,...,28.0,52.0,1.610613e+09,119.0,0.489,1.000,0.389,23.0,47.0,0
3,2022-03-12,22101008.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,122.0,0.484,0.933,...,33.0,55.0,1.610613e+09,109.0,0.413,0.696,0.386,27.0,39.0,1
4,2022-03-12,22101009.0,Final,1.610613e+09,1.610613e+09,2021.0,1.610613e+09,115.0,0.551,0.750,...,32.0,39.0,1.610613e+09,127.0,0.471,0.760,0.387,28.0,50.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31009,2025-05-04 00:00:00,42400157.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,89.0,40.500,71.400,...,14.0,52.0,NaN,103.0,47.600,77.800,41.900,24.0,38.0,0
31010,2025-05-03 00:00:00,42400177.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,120.0,52.200,71.400,...,24.0,46.0,NaN,101.0,47.600,68.200,29.600,30.0,36.0,1
31011,2025-05-02 00:00:00,42400156.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,107.0,41.100,81.800,...,26.0,42.0,NaN,115.0,44.900,71.700,40.000,23.0,46.0,0
31012,2025-05-01 00:00:00,42400126.0,NaN,1.610613e+09,1.610613e+09,2024.0,NaN,113.0,46.900,84.800,...,25.0,36.0,NaN,116.0,47.700,80.000,33.300,18.0,43.0,0


**Add Today's Matchups for Feature Engineering**

In [11]:
if matchups is None:
    print('No games today')
    df_combined = df_old
else:
    df_combined = pd.concat([df_old, df_today], ignore_index = True)
    df_combined

**Data Processing**

In [12]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2022-03-12,22101005.0,1610612748.0,1610612750.0,2021.0,104.0,0.398,0.760,0.333,23.0,53.0,113.0,0.422,0.875,0.357,21.0,46.0,0.0,0,0.0
1,2022-03-12,22101006.0,1610612741.0,1610612739.0,2021.0,101.0,0.443,0.933,0.429,20.0,46.0,91.0,0.419,0.824,0.208,19.0,40.0,1.0,0,1.0
2,2022-03-12,22101007.0,1610612759.0,1610612754.0,2021.0,108.0,0.412,0.813,0.324,28.0,52.0,119.0,0.489,1.000,0.389,23.0,47.0,0.0,0,0.0
3,2022-03-12,22101008.0,1610612744.0,1610612749.0,2021.0,122.0,0.484,0.933,0.400,33.0,55.0,109.0,0.413,0.696,0.386,27.0,39.0,1.0,0,1.0
4,2022-03-12,22101009.0,1610612743.0,1610612761.0,2021.0,115.0,0.551,0.750,0.407,32.0,39.0,127.0,0.471,0.760,0.387,28.0,50.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30990,2025-05-06 00:00:00,42400202.0,1610612739.0,1610612754.0,2024.0,119.0,43.300,85.700,28.200,24.0,43.0,120.0,51.800,72.400,39.300,28.0,44.0,0.0,1,0.0
30991,2025-05-06 00:00:00,42400231.0,1610612750.0,1610612744.0,2024.0,88.0,39.500,88.200,17.200,19.0,41.0,99.0,39.100,86.700,42.900,26.0,51.0,0.0,1,0.0
31002,2025-05-07 00:00:00,42400212.0,1610612738.0,1610612752.0,2024.0,90.0,36.200,80.000,25.000,17.0,51.0,91.0,43.000,80.000,29.000,19.0,46.0,0.0,1,0.0
31003,2025-05-07 00:00:00,42400222.0,1610612760.0,1610612743.0,2024.0,149.0,56.200,91.700,44.400,31.0,44.0,106.0,37.900,78.100,40.500,18.0,38.0,1.0,1,1.0


**Feature Engineering**

In [13]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)



#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')

# save file
df_combined.to_csv(DATAPATH / 'games_engineered.csv', index=False)


df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28 00:00:00+00:00,20300003,1610612747,1610612742,2003,109,0.505859,0.600098,0.350098,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28 00:00:00+00:00,20300002,1610612759,1610612756,2003,83,0.425049,0.769043,0.099976,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28 00:00:00+00:00,20300001,1610612755,1610612748,2003,89,0.439941,0.533203,0.350098,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29 00:00:00+00:00,20300006,1610612740,1610612737,2003,88,0.323975,0.700195,0.160034,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29 00:00:00+00:00,20300008,1610612765,1610612754,2003,87,0.392090,0.742188,0.333008,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27598,2025-05-06 00:00:00+00:00,42400231,1610612750,1610612744,2024,88,39.500000,88.187500,17.203125,19,...,-0.497656,0.222396,-2.333333,-1.285714,-3.0,-2.933333,4.000000,6.285714,5.2,3.866667
27599,2025-05-06 00:00:00+00:00,42400202,1610612739,1610612754,2024,119,43.312500,85.687500,28.203125,24,...,-1.567188,2.481250,-2.333333,-5.000000,-3.4,-0.066667,3.666667,2.714286,3.7,1.400000
27600,2025-05-07 00:00:00+00:00,42400222,1610612760,1610612743,2024,149,56.187500,91.687500,44.406250,31,...,-1.096875,-2.479167,2.333333,3.000000,1.8,-1.666667,-9.666667,1.142857,-2.3,-1.000000
27601,2025-05-07 00:00:00+00:00,42400212,1610612738,1610612752,2024,90,36.187500,80.000000,25.000000,17,...,1.145312,3.023958,3.000000,0.428571,1.2,0.266667,0.666667,4.285714,3.0,2.266667


**Process Data for Convenient Dashboarding**

In [14]:
processor = NBADataProcessor()
exported_files = processor.export_data_for_dashboard()

print("\nData Processing Complete!")
print(f"Files exported for Dashboards:")
for key, value in exported_files.items():
    if value:
        print(f"- {key}: {value}")

2025-05-08 07:09:18,374 INFO: Initialized NBADataProcessor with data_path=data, model_path=models
2025-05-08 07:09:18,375 INFO: Exporting data for dashboards to data
2025-05-08 07:09:18,376 INFO: Preparing today's games data
2025-05-08 07:09:18,377 INFO: Loading data from data\games_engineered.csv
2025-05-08 07:09:19,019 INFO: Successfully loaded data: 27603 rows, 245 columns
2025-05-08 07:09:19,020 INFO: Filtering for season: 2024
2025-05-08 07:09:19,022 INFO: Processing data for prediction
2025-05-08 07:09:19,057 INFO: Making predictions
2025-05-08 07:09:19,058 WARNING: Model not loaded, loading now...
2025-05-08 07:09:19,058 INFO: Loading model from models\model.pkl


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

2025-05-08 07:09:20,010 INFO: Successfully loaded model
2025-05-08 07:09:20,011 INFO: Removing unused features
2025-05-08 07:09:20,401 INFO: Successfully made predictions for 1 games
2025-05-08 07:09:20,403 INFO: Prepared predictions for 1 games today
2025-05-08 07:09:20,405 INFO: Preparing processed games data
2025-05-08 07:09:20,406 INFO: Loading data from data\games_engineered.csv


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2025-05-08 07:09:21,027 INFO: Successfully loaded data: 27603 rows, 245 columns
2025-05-08 07:09:21,028 INFO: Filtering for season: 2024
2025-05-08 07:09:21,029 INFO: Processing data for prediction
2025-05-08 07:09:21,081 INFO: Making predictions
2025-05-08 07:09:21,081 INFO: Removing unused features
2025-05-08 07:09:21,219 INFO: Successfully made predictions for 599 games
2025-05-08 07:09:21,225 INFO: Calculating daily running accuracy metrics


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

2025-05-08 07:09:21,231 INFO: Calculating team-specific daily running accuracy
2025-05-08 07:09:21,278 INFO: Calculating home/away daily running accuracy
2025-05-08 07:09:21,322 INFO: Calculating overall daily running accuracy
2025-05-08 07:09:21,406 INFO: Adding running accuracy metrics to games dataframe
2025-05-08 07:09:21,420 INFO: Calculating team-specific prediction accuracy (legacy method)
2025-05-08 07:09:21,450 INFO: Calculated team accuracy for 30 teams
2025-05-08 07:09:21,450 INFO: Processed 599 completed games with 366 correct predictions
2025-05-08 07:09:21,456 INFO: Filtering columns for dashboard
2025-05-08 07:09:21,459 INFO: Filtered dataframe from 257 to 22 columns
2025-05-08 07:09:21,465 INFO: Exported filtered games data to data\games_dashboard.csv
2025-05-08 07:09:21,465 INFO: Exporting running accuracy metrics
2025-05-08 07:09:21,527 INFO: Exported running accuracy metrics to data\running_accuracy_metrics.csv
2025-05-08 07:09:21,528 INFO: Exported season summary to

**Upload to Google Drive**

In [15]:

files_to_upload = [
    DATAPATH / 'games_dashboard.csv',
    DATAPATH / 'season_summary_stats.csv',
    DATAPATH / 'running_accuracy_metrics.csv'
]
upload_to_drive(files_to_upload, GOOGLE_FOLDER_ID)

2025-05-08 07:09:21,549 INFO: file_cache is only supported with oauth2client<4.0.0
Found 3 existing files in the folder
File 'games_dashboard.csv' updated successfully. File ID: 1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46
File 'season_summary_stats.csv' updated successfully. File ID: 1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S
File 'running_accuracy_metrics.csv' updated successfully. File ID: 18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti


['1g8Mc2SQafXApx8pwPBPhol2bDXiNOT46',
 '1zeqYZGhbfFj5kmj5ZRI1qXL2HKnU3B8S',
 '18Hb24olk1Y4iroj55CEvDSxKXEGof4Ti']